In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim

from src.models import CifarResNet, MNIST_CNN, CIFAR_CNN
from src.helpers import evaluate_rob_accuracy, evaluate_clean_accuracy, load_model, safe_model,_evaluate_model
from src.data_loader import load_torchvision_dataset, load_imagenette
#from src.pruning import identify_layers, _evaluate_sparsity

import time

if torch.cuda.is_available() == True:
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
print(device)
dtype = torch.float32

cuda:0


In [1]:
import pandas as pd

In [4]:
pd.read_pickle('./results/preliminary-double.pkl')

,l2_robustness,linf_robustness,clean_accuracy
1,70.117186,67.382809,67.90
2,68.945312,66.015623,69.22
4,68.554687,64.062499,68.67
8,65.820310,62.499996,65.39
16,57.226559,52.343747,55.59


In [18]:

evaluate_clean_accuracy(model.to(device), test_loader, device)

(10.76, 0.0)

In [10]:
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD, FGSM, L0BrendelBethgeAttack, L2CarliniWagnerAttack

epochs = 500


def run(training_method):
    model = CIFAR_CNN().to(device)
    #print(model.device)
    compression_rates = [1,2,4,8,16]
    stats = {}
    for ratio in compression_rates:
        print('compression rate: ', 1-1/ratio)
        fit = get_train_method(model, training_method)
        model.prune_magnitude_global_unstruct(1-1/ratio, device)
        #print(fit)
        train_data = fit(train_loader, test_loader, epochs, device, eps=8/255, patience=5)
        images, labels = next(iter(test_loader))
        images, labels = images.to(device), labels.to(device)
        #stats[f'ratio']['l0_robustness'] = bb_attack(model, images, labels).item()
        #print('bb done')
        stats[f'{ratio}'] = {}
        stats[f'{ratio}']['l2_robustness'] = cw_attack(model, images, labels).item()
        print('cw done')
        stats[f'{ratio}']['linf_robustness'] = pgd_attack(model, images, labels).item()
        print('pgd done')
        stats[f'{ratio}']['clean_accuracy'] = train_data['val_accuracy']
        
    return(stats)
        
        
        

def get_train_method(model, method):
    if method=='standard':
        return model.fit
    if method=='free':
        return model.fit_free
    if method=='fast':
        return model.fit_fast
    if method=='fast_double':
        return model.fit_fast_with_double_update

def bb_attack(model, images, labels, eps=8/255):
    model.eval()
    fmodel = PyTorchModel(model, bounds=(0, 1))
    attack = L0BrendelBethgeAttack()
    raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=eps)
    model.train()

    return (1 - torch.sum(success)/len(success)) / 100

def cw_attack(model, images, labels, eps=8/255):
    model.eval()
    fmodel = PyTorchModel(model, bounds=(0, 1))
    attack = L2CarliniWagnerAttack()
    raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=eps)
    model.train()

    return (1 - torch.sum(success)/len(success)) / 100

def pgd_attack(model, images, labels, eps=8/255):
    model.eval()
    fmodel = PyTorchModel(model, bounds=(0, 1))
    attack = LinfPGD()
    raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=eps)
    model.train()

    return (1 - torch.sum(success)/len(success)) / 100

# Double Update vs. Single Update Fast Adversarial Training
Specs:

CIFAR CNN: 4 Conv (16,16,32,32) with batchnorm, 2 FC (128,10)

Data: Cifar (32,32,3)

30 Epochs with eps=8/255



Standard Fast Adv. Training:
Clean: 63.05%
Robust: 59.34%

Fast Adv. Training w Double Update:
Clean: 63.99%
Robust: 61.35%




# Initialization

In [11]:
#model = ResNet()
#model = MNIST_CNN()
model = CIFAR_CNN()

identifying layers


In [12]:
train_loader, test_loader = load_torchvision_dataset('CIFAR10')

Files already downloaded and verified
Files already downloaded and verified


In [11]:
PATH = './saved-models/CIFAR-baseline-150-epochs.pth'
model = load_model(model, PATH)

# Experiment


1. Prune
2. Train
3. measure robust accuracy


In [13]:
standard_stats = run('standard')

identifying layers
compression rate:  0.0
[1,     1] loss: 3.07543, train_accuracy: 8.59
[1,     2] loss: 2.74960, train_accuracy: 11.13
[1,     3] loss: 2.46489, train_accuracy: 17.19
[1,     4] loss: 2.45922, train_accuracy: 21.09
[1,     5] loss: 2.41755, train_accuracy: 20.90
[1,     6] loss: 2.31154, train_accuracy: 22.66
[1,     7] loss: 2.24753, train_accuracy: 22.66
[1,     8] loss: 2.26820, train_accuracy: 20.70
[1,     9] loss: 2.13691, train_accuracy: 26.95
[1,    10] loss: 2.09834, train_accuracy: 24.80
[1,    11] loss: 2.12638, train_accuracy: 28.32
[1,    12] loss: 2.06679, train_accuracy: 31.25
[1,    13] loss: 2.05545, train_accuracy: 27.34
[1,    14] loss: 1.94770, train_accuracy: 33.20
[1,    15] loss: 1.88556, train_accuracy: 34.38
[1,    16] loss: 1.91349, train_accuracy: 31.64
[1,    17] loss: 1.89596, train_accuracy: 32.62
[1,    18] loss: 1.91558, train_accuracy: 31.45
[1,    19] loss: 1.85688, train_accuracy: 34.96
[1,    20] loss: 1.89446, train_accuracy: 38.09

[2,    72] loss: 1.25887, train_accuracy: 53.71
[2,    73] loss: 1.26233, train_accuracy: 55.66
[2,    74] loss: 1.21899, train_accuracy: 56.45
[2,    75] loss: 1.30196, train_accuracy: 54.69
[2,    76] loss: 1.26409, train_accuracy: 56.45
[2,    77] loss: 1.27879, train_accuracy: 53.32
[2,    78] loss: 1.30535, train_accuracy: 53.71
[2,    79] loss: 1.21662, train_accuracy: 53.32
[2,    80] loss: 1.31086, train_accuracy: 55.66
[2,    81] loss: 1.26724, train_accuracy: 54.10
[2,    82] loss: 1.20118, train_accuracy: 57.23
[2,    83] loss: 1.19477, train_accuracy: 58.40
[2,    84] loss: 1.27326, train_accuracy: 52.93
[2,    85] loss: 1.33639, train_accuracy: 53.71
[2,    86] loss: 1.32417, train_accuracy: 52.15
[2,    87] loss: 1.21581, train_accuracy: 53.52
[2,    88] loss: 1.30643, train_accuracy: 53.12
[2,    89] loss: 1.37992, train_accuracy: 52.15
[2,    90] loss: 1.22989, train_accuracy: 55.27
[2,    91] loss: 1.25408, train_accuracy: 55.08
[2,    92] loss: 1.20920, train_accuracy

[4,    42] loss: 1.03705, train_accuracy: 62.89
[4,    43] loss: 1.07753, train_accuracy: 63.28
[4,    44] loss: 1.02250, train_accuracy: 63.87
[4,    45] loss: 0.98614, train_accuracy: 65.43
[4,    46] loss: 0.99279, train_accuracy: 66.60
[4,    47] loss: 0.95851, train_accuracy: 65.23
[4,    48] loss: 1.06056, train_accuracy: 64.84
[4,    49] loss: 1.00350, train_accuracy: 64.26
[4,    50] loss: 0.94727, train_accuracy: 65.62
[4,    51] loss: 1.05110, train_accuracy: 64.45
[4,    52] loss: 0.99912, train_accuracy: 63.09
[4,    53] loss: 1.02604, train_accuracy: 63.67
[4,    54] loss: 1.04325, train_accuracy: 61.13
[4,    55] loss: 1.05767, train_accuracy: 61.52
[4,    56] loss: 1.04199, train_accuracy: 61.72
[4,    57] loss: 1.01788, train_accuracy: 62.30
[4,    58] loss: 1.06780, train_accuracy: 62.89
[4,    59] loss: 1.04485, train_accuracy: 64.26
[4,    60] loss: 1.05533, train_accuracy: 62.30
[4,    61] loss: 0.97149, train_accuracy: 66.80
[4,    62] loss: 0.99877, train_accuracy

[6,    12] loss: 0.87107, train_accuracy: 70.51
[6,    13] loss: 0.89852, train_accuracy: 67.19
[6,    14] loss: 0.86627, train_accuracy: 70.90
[6,    15] loss: 0.84164, train_accuracy: 70.70
[6,    16] loss: 0.79746, train_accuracy: 71.09
[6,    17] loss: 0.83180, train_accuracy: 68.55
[6,    18] loss: 0.80388, train_accuracy: 71.29
[6,    19] loss: 0.92591, train_accuracy: 66.80
[6,    20] loss: 0.86026, train_accuracy: 68.36
[6,    21] loss: 0.85710, train_accuracy: 70.12
[6,    22] loss: 0.77981, train_accuracy: 71.68
[6,    23] loss: 0.96540, train_accuracy: 66.60
[6,    24] loss: 0.88558, train_accuracy: 68.55
[6,    25] loss: 0.86126, train_accuracy: 70.51
[6,    26] loss: 0.81775, train_accuracy: 73.24
[6,    27] loss: 0.80494, train_accuracy: 70.51
[6,    28] loss: 0.90528, train_accuracy: 69.34
[6,    29] loss: 0.90150, train_accuracy: 68.36
[6,    30] loss: 0.85550, train_accuracy: 70.31
[6,    31] loss: 0.83009, train_accuracy: 73.05
[6,    32] loss: 0.88334, train_accuracy

[7,    84] loss: 0.80167, train_accuracy: 72.07
[7,    85] loss: 0.89240, train_accuracy: 67.97
[7,    86] loss: 0.87248, train_accuracy: 70.12
[7,    87] loss: 0.78617, train_accuracy: 73.83
[7,    88] loss: 0.77728, train_accuracy: 74.02
[7,    89] loss: 0.78611, train_accuracy: 70.12
[7,    90] loss: 0.77112, train_accuracy: 73.44
[7,    91] loss: 0.73884, train_accuracy: 72.66
[7,    92] loss: 0.80555, train_accuracy: 68.55
[7,    93] loss: 0.89909, train_accuracy: 66.99
[7,    94] loss: 0.80103, train_accuracy: 73.63
[7,    95] loss: 0.81962, train_accuracy: 71.68
[7,    96] loss: 0.84272, train_accuracy: 68.55
[7,    97] loss: 0.85348, train_accuracy: 71.09
[7,    98] loss: 0.89267, train_accuracy: 69.94
duration: 11 s - train loss: 0.79816 - train accuracy: 72.18 - validation loss: 1.09 - validation accuracy: 62.13 
[8,     1] loss: 0.73178, train_accuracy: 76.37
[8,     2] loss: 0.75952, train_accuracy: 72.85
[8,     3] loss: 0.70229, train_accuracy: 75.59
[8,     4] loss: 0.75

...

[16,    96] loss: 0.84176, train_accuracy: 69.92
[16,    97] loss: 0.92499, train_accuracy: 69.14
[16,    98] loss: 0.83059, train_accuracy: 70.54
duration: 24 s - train loss: 0.86072 - train accuracy: 69.54 - validation loss: 1.08 - validation accuracy: 62.31 
[17,     1] loss: 0.82085, train_accuracy: 68.55
[17,     2] loss: 0.76267, train_accuracy: 73.05
[17,     3] loss: 0.77627, train_accuracy: 74.02
[17,     4] loss: 0.79598, train_accuracy: 72.66
[17,     5] loss: 0.81698, train_accuracy: 71.29
[17,     6] loss: 0.85135, train_accuracy: 71.09
[17,     7] loss: 0.89824, train_accuracy: 67.38
[17,     8] loss: 0.81004, train_accuracy: 72.27
[17,     9] loss: 0.76715, train_accuracy: 72.46
[17,    10] loss: 0.79472, train_accuracy: 73.63
[17,    11] loss: 0.79862, train_accuracy: 72.27
[17,    12] loss: 0.94414, train_accuracy: 66.02
[17,    13] loss: 0.83574, train_accuracy: 70.31
[17,    14] loss: 0.82137, train_accuracy: 71.29
[17,    15] loss: 0.82732, train_accuracy: 70.90
[17

[18,    63] loss: 0.81799, train_accuracy: 71.29
[18,    64] loss: 0.84422, train_accuracy: 67.19
[18,    65] loss: 0.86432, train_accuracy: 69.92
[18,    66] loss: 0.79836, train_accuracy: 71.29
[18,    67] loss: 0.85918, train_accuracy: 70.70
[18,    68] loss: 0.87735, train_accuracy: 70.12
[18,    69] loss: 0.81942, train_accuracy: 70.31
[18,    70] loss: 0.92133, train_accuracy: 69.53
[18,    71] loss: 0.82966, train_accuracy: 71.09
[18,    72] loss: 0.85452, train_accuracy: 69.73
[18,    73] loss: 0.88909, train_accuracy: 67.19
[18,    74] loss: 0.88987, train_accuracy: 71.09
[18,    75] loss: 0.92115, train_accuracy: 66.80
[18,    76] loss: 0.81115, train_accuracy: 70.12
[18,    77] loss: 0.87563, train_accuracy: 70.90
[18,    78] loss: 0.82587, train_accuracy: 70.70
[18,    79] loss: 0.85220, train_accuracy: 69.53
[18,    80] loss: 0.79867, train_accuracy: 71.48
[18,    81] loss: 0.92477, train_accuracy: 66.99
[18,    82] loss: 0.81414, train_accuracy: 70.12
[18,    83] loss: 0.

[20,    30] loss: 0.82383, train_accuracy: 72.27
[20,    31] loss: 0.79074, train_accuracy: 71.09
[20,    32] loss: 0.79394, train_accuracy: 70.12
[20,    33] loss: 0.86947, train_accuracy: 69.34
[20,    34] loss: 0.86101, train_accuracy: 69.53
[20,    35] loss: 0.81953, train_accuracy: 72.85
[20,    36] loss: 0.81717, train_accuracy: 69.53
[20,    37] loss: 0.90287, train_accuracy: 67.38
[20,    38] loss: 0.77697, train_accuracy: 72.66
[20,    39] loss: 0.81861, train_accuracy: 70.12
[20,    40] loss: 0.92206, train_accuracy: 67.58
[20,    41] loss: 0.79143, train_accuracy: 73.83
[20,    42] loss: 0.82477, train_accuracy: 71.09
[20,    43] loss: 0.89041, train_accuracy: 69.53
[20,    44] loss: 0.87090, train_accuracy: 68.36
[20,    45] loss: 0.73865, train_accuracy: 75.00
[20,    46] loss: 0.85140, train_accuracy: 70.70
[20,    47] loss: 0.82297, train_accuracy: 72.85
[20,    48] loss: 0.81319, train_accuracy: 71.09
[20,    49] loss: 0.83570, train_accuracy: 70.31
[20,    50] loss: 0.

[21,    97] loss: 0.85645, train_accuracy: 67.38
[21,    98] loss: 0.81864, train_accuracy: 71.73
duration: 25 s - train loss: 0.84274 - train accuracy: 70.21 - validation loss: 1.08 - validation accuracy: 62.59 
[22,     1] loss: 0.88381, train_accuracy: 67.97
[22,     2] loss: 0.82424, train_accuracy: 70.31
[22,     3] loss: 0.80804, train_accuracy: 71.48
[22,     4] loss: 0.83496, train_accuracy: 70.12
[22,     5] loss: 0.78535, train_accuracy: 70.90
[22,     6] loss: 0.88623, train_accuracy: 67.19
[22,     7] loss: 0.82410, train_accuracy: 71.29
[22,     8] loss: 0.76520, train_accuracy: 72.85
[22,     9] loss: 0.82453, train_accuracy: 69.92
[22,    10] loss: 0.87297, train_accuracy: 69.14
[22,    11] loss: 0.91808, train_accuracy: 68.16
[22,    12] loss: 0.85648, train_accuracy: 68.75
[22,    13] loss: 0.84969, train_accuracy: 70.90
[22,    14] loss: 0.76880, train_accuracy: 73.83
[22,    15] loss: 0.78363, train_accuracy: 70.12
[22,    16] loss: 0.85905, train_accuracy: 71.29
[22

[23,    64] loss: 0.76802, train_accuracy: 73.83
[23,    65] loss: 0.87542, train_accuracy: 68.36
[23,    66] loss: 0.77040, train_accuracy: 72.27
[23,    67] loss: 0.90749, train_accuracy: 66.80
[23,    68] loss: 0.84779, train_accuracy: 69.34
[23,    69] loss: 0.84901, train_accuracy: 69.53
[23,    70] loss: 0.86084, train_accuracy: 68.55
[23,    71] loss: 0.94943, train_accuracy: 65.04
[23,    72] loss: 0.82837, train_accuracy: 70.12
[23,    73] loss: 0.90126, train_accuracy: 68.36
[23,    74] loss: 0.83996, train_accuracy: 71.09
[23,    75] loss: 0.86564, train_accuracy: 70.51
[23,    76] loss: 0.88117, train_accuracy: 70.12
[23,    77] loss: 0.87036, train_accuracy: 69.14
[23,    78] loss: 0.82191, train_accuracy: 72.27
[23,    79] loss: 0.82629, train_accuracy: 71.48
[23,    80] loss: 0.81973, train_accuracy: 70.31
[23,    81] loss: 0.86298, train_accuracy: 67.19
[23,    82] loss: 0.88396, train_accuracy: 70.70
[23,    83] loss: 0.87345, train_accuracy: 68.95
[23,    84] loss: 0.

In [7]:
standard_stats['1']['l2_robustness']

0.55078125

In [14]:
import pandas as pd
df = pd.DataFrame(data=standard_stats).T
df['l2_robustness'] = df['l2_robustness']*10000
df['linf_robustness'] = df['linf_robustness']*10000
df.to_pickle('./results/preliminary-standard-no-augmentation.pkl')
df

,l2_robustness,linf_robustness,clean_accuracy
1,62.109372,53.906250,62.07
2,59.570312,52.343747,62.11
4,62.890626,58.007813,62.30
8,65.820310,58.789062,63.78
16,64.843749,56.249998,62.44


In [6]:
fast_stats = run('fast')

identifying layers
[1,     1] loss: 2.95128, train_accuracy: 10.35
[1,    11] loss: 2.32203, train_accuracy: 20.90
[1,    21] loss: 2.07043, train_accuracy: 25.78
[1,    31] loss: 2.01518, train_accuracy: 26.56
[1,    41] loss: 2.03410, train_accuracy: 25.59
[1,    51] loss: 1.96823, train_accuracy: 27.73
[1,    61] loss: 2.02681, train_accuracy: 29.30
[1,    71] loss: 1.99444, train_accuracy: 30.27
[1,    81] loss: 1.97469, train_accuracy: 28.91
[1,    91] loss: 1.94731, train_accuracy: 29.30
duration: 60 s - train loss: 2.02881 - train accuracy: 27.44 - validation loss: 1.65035 - validation accuracy: 40.17 
[2,     1] loss: 1.80543, train_accuracy: 33.79
[2,    11] loss: 1.87866, train_accuracy: 33.98
[2,    21] loss: 1.83539, train_accuracy: 33.59
[2,    31] loss: 1.79033, train_accuracy: 33.59
[2,    41] loss: 1.78200, train_accuracy: 35.35
[2,    51] loss: 1.79811, train_accuracy: 32.62
[2,    61] loss: 1.79824, train_accuracy: 35.55
[2,    71] loss: 1.75369, train_accuracy: 37.11

KeyboardInterrupt: 

In [ ]:
free_stats = run('free')

identifying layers
[1,     1] loss: 4.04980, train_accuracy: 4.44
[1,     2] loss: 3.13854, train_accuracy: 5.86
[1,     3] loss: 2.63508, train_accuracy: 10.21
[1,     4] loss: 2.44655, train_accuracy: 15.32
[1,     5] loss: 2.31197, train_accuracy: 18.89
[1,     6] loss: 2.36261, train_accuracy: 16.88
[1,     7] loss: 2.33470, train_accuracy: 19.06
[1,     8] loss: 2.32105, train_accuracy: 18.72
[1,     9] loss: 2.27571, train_accuracy: 18.08
[1,    10] loss: 2.23458, train_accuracy: 20.93
[1,    11] loss: 2.20737, train_accuracy: 20.59
[1,    12] loss: 2.24530, train_accuracy: 20.59
[1,    13] loss: 2.24561, train_accuracy: 17.91
[1,    14] loss: 2.16499, train_accuracy: 22.10
[1,    15] loss: 2.18761, train_accuracy: 21.57
[1,    16] loss: 2.17943, train_accuracy: 19.81
[1,    17] loss: 2.14748, train_accuracy: 21.29
[1,    18] loss: 2.14471, train_accuracy: 21.15
[1,    19] loss: 2.21635, train_accuracy: 20.65
[1,    20] loss: 2.14575, train_accuracy: 20.34
[1,    21] loss: 2.1973

[2,    71] loss: 1.96260, train_accuracy: 29.72
[2,    72] loss: 1.92929, train_accuracy: 29.77
[2,    73] loss: 1.94031, train_accuracy: 28.85
[2,    74] loss: 1.90231, train_accuracy: 28.66
[2,    75] loss: 2.04087, train_accuracy: 23.94
[2,    76] loss: 1.95751, train_accuracy: 28.21
[2,    77] loss: 1.91723, train_accuracy: 27.40
[2,    78] loss: 1.96543, train_accuracy: 28.93
[2,    79] loss: 1.98063, train_accuracy: 24.89
[2,    80] loss: 1.90134, train_accuracy: 28.12
[2,    81] loss: 1.92274, train_accuracy: 28.99
[2,    82] loss: 1.98986, train_accuracy: 27.26
[2,    83] loss: 1.97384, train_accuracy: 26.56
[2,    84] loss: 1.96078, train_accuracy: 27.96
[2,    85] loss: 1.92282, train_accuracy: 28.40
[2,    86] loss: 1.90109, train_accuracy: 28.52
[2,    87] loss: 1.92160, train_accuracy: 30.44
[2,    88] loss: 1.90432, train_accuracy: 31.19
[2,    89] loss: 1.90042, train_accuracy: 29.83
[2,    90] loss: 1.86666, train_accuracy: 32.48
[2,    91] loss: 1.94562, train_accuracy

In [ ]:
double_stats = run('fast_double')

In [7]:
images, labels = next(iter(test_loader))
images, labels = images.to(device), labels.to(device)


In [8]:
bb_attack(model, images, labels)

1
2


KeyboardInterrupt: 

In [9]:
pgd_attack(model, images, labels, eps=8/255)

tensor(0.3359, device='cuda:0')

In [10]:
cw_attack(model, images, labels, eps=8/255)

tensor(0.2656, device='cuda:0')

In [13]:
bb_attack(model, images, labels)

NameError: name 'L0BrendelBethgeAttackfoolbox' is not defined

# Test

In [4]:
model.fit_free(train_loader, test_loader, 1, device, number_of_replays=3, eps = 8/255)

[1,     1] loss: 4.13015, train_accuracy: 4.17
[1,     2] loss: 3.66484, train_accuracy: 4.88
[1,     3] loss: 3.12940, train_accuracy: 6.97
[1,     4] loss: 2.90560, train_accuracy: 8.98
[1,     5] loss: 2.72787, train_accuracy: 9.70
[1,     6] loss: 2.53809, train_accuracy: 10.22
[1,     7] loss: 2.40960, train_accuracy: 15.62
[1,     8] loss: 2.43231, train_accuracy: 14.13
[1,     9] loss: 2.44054, train_accuracy: 14.78
[1,    10] loss: 2.36526, train_accuracy: 17.90
[1,    11] loss: 2.37196, train_accuracy: 19.01
[1,    12] loss: 2.28873, train_accuracy: 18.49
[1,    13] loss: 2.31310, train_accuracy: 14.58
[1,    14] loss: 2.30206, train_accuracy: 18.55
[1,    15] loss: 2.27567, train_accuracy: 15.89
[1,    16] loss: 2.21592, train_accuracy: 18.03
[1,    17] loss: 2.19046, train_accuracy: 20.05
[1,    18] loss: 2.15967, train_accuracy: 23.11
[1,    19] loss: 2.23406, train_accuracy: 18.68
[1,    20] loss: 2.25862, train_accuracy: 17.45
[1,    21] loss: 2.12274, train_accuracy: 23.

{'criterion': CrossEntropyLoss(),
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.001
     weight_decay: 0
 ),
 'hist': 'Not implemented',
 'val_accuracy': 34.82}

In [5]:
model.fit_fast(train_loader, test_loader, 1, device,eps = 8/255)

[1,     1] loss: 1.96565, train_accuracy: 28.91
[1,    11] loss: 1.97713, train_accuracy: 27.93
[1,    21] loss: 1.93998, train_accuracy: 27.34
[1,    31] loss: 1.85866, train_accuracy: 31.05
[1,    41] loss: 1.93476, train_accuracy: 29.88
[1,    51] loss: 1.81361, train_accuracy: 33.79
[1,    61] loss: 1.79394, train_accuracy: 34.96
[1,    71] loss: 1.87157, train_accuracy: 32.42
[1,    81] loss: 1.81604, train_accuracy: 33.98
[1,    91] loss: 1.78834, train_accuracy: 31.84
duration: 255 s - train loss: 1.84690 - train accuracy: 32.50 - validation loss: 1.53757 - validation accuracy: 44.68 
Finished Training


{'criterion': CrossEntropyLoss(),
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.001
     weight_decay: 0
 ),
 'hist': 'Not implemented',
 'val_accuracy': 44.68}

In [6]:
model.fit_fast_with_double_update(train_loader, test_loader, 1, device,eps = 8/255)

[1,     1] loss: 1.70809, train_accuracy: 39.65
[1,    11] loss: 1.82264, train_accuracy: 33.59
[1,    21] loss: 1.73971, train_accuracy: 32.81
[1,    31] loss: 1.81496, train_accuracy: 35.94
[1,    41] loss: 1.78697, train_accuracy: 31.45
[1,    51] loss: 1.78453, train_accuracy: 33.20
[1,    61] loss: 1.72529, train_accuracy: 35.55
[1,    71] loss: 1.67665, train_accuracy: 38.67
[1,    81] loss: 1.74809, train_accuracy: 35.55
[1,    91] loss: 1.71799, train_accuracy: 38.67
duration: 320 s - train loss: 1.76333 - train accuracy: 35.67 - validation loss: 1.45301 - validation accuracy: 48.06 
Finished Training


{'criterion': CrossEntropyLoss(),
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.001
     weight_decay: 0
 ),
 'hist': 'Not implemented',
 'val_accuracy': 48.06}

In [ ]:
losses = [10, 9, 8,7,6,5,4,3,2,2]
patience = 3
check_early_stopping(losses, patience)

In [17]:
a = 4
b = [5,6,7]
list(filter(lambda x: x<a, b))

[]

In [20]:
b[-1:]

[7]

In [ ]:
def check_early_stopping(val_loss_hist, patience):
    filter(lambda x: val_loss_hist[-1:] > x, val_loss_hist[patience:])
    return list(filter(lambda x: val_loss_hist[-1:][0] > x, val_loss_hist[patience:]))